In [3]:
from sklearn.pipeline import Pipeline
from functions import *
from sklearn.naive_bayes import MultinomialNB 
from sklearn.metrics import confusion_matrix
from nltk.corpus import stopwords
from functions import * 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import sklearn
from matplotlib import pyplot as plt
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import pandas as pd
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
import random
from sklearn.model_selection import cross_validate as cvt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier as dtc
from copy import deepcopy as dp

def tokenize(text):
    return text.split(' ')

In [20]:
test, train = load_pkl(r"C:\Users\HEndo\Documents\GitHub\zemi\cleaned_corpus\augmented_corpus.pkl")
test_a, train_a = load_pkl(r"C:\Users\HEndo\Documents\GitHub\zemi\cleaned_corpus\augmented_org_ans.pkl")
train_X = dp(train)
train_Y = dp(train_a)
sw = stopwords.words('english') + ['hes']


In [7]:
import optuna

In [8]:
import warnings
warnings.filterwarnings('ignore')
optuna.logging.enable_default_handler()


In [18]:
def objective(trial):
    params = {
    '_min': trial.suggest_uniform('_min', 1e-3, 8*1e-3),
    '_max': trial.suggest_uniform('_max', 0.7, 0.9),
    'mx_depth': trial.suggest_discrete_uniform('mx_depth', 10, 500, 10),
    'mn_sp': trial.suggest_uniform('mn_sp', 0.1, 0.3),
    'mx_feat': trial.suggest_uniform('mx_feat', 0.8, 1.0)
    }
    data = list(zip(train_X, train_Y))
    random.shuffle(data)
    ff_train, ff_train_a = zip(*data)
    ff_train = [' '.join(text) for text in ff_train]
    vect = TfidfVectorizer(stop_words=sw,tokenizer=tokenize, 
                          min_df= params['_min'], max_df=params['_max'], 
                          ngram_range=(1,2))
    tfidf = vect.fit_transform(ff_train)
    tree = dtc(max_depth=params['mx_depth'], 
               min_samples_split= params['mn_sp'],
               max_features= params['mx_feat']
              )

    scores = cvt(tree, tfidf, ff_train_a, cv = 4, return_train_score= True, scoring = 'f1_macro', )
    test_s = scores['test_score'].mean()
    train_s = scores['train_score'].mean()
    f_score = 1.0 - float(test_s) + float(abs(train_s-test_s))/2.0
    return f_score

In [21]:
study = optuna.create_study()
study.optimize(objective, n_trials=500, n_jobs = -1, verb_pace=100)

[I 2019-02-27 14:48:12,677] Finished trial 100 / 500. Current best value is 0.21809442208226226 with parameters: {'_min': 0.0010076069969777687, '_max': 0.893393602554536, 'mx_depth': 80.0, 'mn_sp': 0.1550354937299351, 'mx_feat': 0.8211110248572838}.
[I 2019-02-27 14:48:13,940] Finished trial 100 / 500. Current best value is 0.21809442208226226 with parameters: {'_min': 0.0010076069969777687, '_max': 0.893393602554536, 'mx_depth': 80.0, 'mn_sp': 0.1550354937299351, 'mx_feat': 0.8211110248572838}.
[I 2019-02-27 14:48:15,042] Finished trial 100 / 500. Current best value is 0.21809442208226226 with parameters: {'_min': 0.0010076069969777687, '_max': 0.893393602554536, 'mx_depth': 80.0, 'mn_sp': 0.1550354937299351, 'mx_feat': 0.8211110248572838}.
[I 2019-02-27 14:49:54,561] Finished trial 200 / 500. Current best value is 0.21809442208226226 with parameters: {'_min': 0.0010076069969777687, '_max': 0.893393602554536, 'mx_depth': 80.0, 'mn_sp': 0.1550354937299351, 'mx_feat': 0.821111024857283

In [30]:
def objective2(trial):
    params = {
    'mx_depth': trial.suggest_discrete_uniform('mx_depth', 10, 100, 10),
    }
    data = list(zip(train_X, train_Y))
    random.shuffle(data)
    ff_train, ff_train_a = zip(*data)
    ff_train = [' '.join(text) for text in ff_train]
    vect = TfidfVectorizer(stop_words=sw,tokenizer=tokenize, 
                          min_df= 0.001, max_df=0.9, 
                          ngram_range=(1,2))
    tfidf = vect.fit_transform(ff_train)
    tree = dtc(max_depth=abs(params['mx_depth']), 
               min_samples_split= 0.15,
               max_features= 0.82
              )

    scores = cvt(tree, tfidf, ff_train_a, cv = 4, return_train_score= True, scoring = 'f1_macro', )
    test_s = scores['test_score'].mean()
    train_s = scores['train_score'].mean()
    f_score = 1.0 - float(test_s) + float(abs(train_s-test_s))/2.0
    
    return f_score

In [32]:
study2 = optuna.create_study()
study2.optimize(objective3, n_trials=25, n_jobs = -1, verb_pace = 10)


[I 2019-02-27 15:02:45,784] Finished trial 10 / 25. Current best value is 0.3113213345048802 with parameters: {'mx_depth': 10.0}.
[I 2019-02-27 15:02:46,984] Finished trial 10 / 25. Current best value is 0.21866960924507006 with parameters: {'mx_depth': 90.0}.
[I 2019-02-27 15:02:55,453] Finished trial 20 / 25. Current best value is 0.21866960924507006 with parameters: {'mx_depth': 90.0}.
[I 2019-02-27 15:02:56,591] Finished trial 20 / 25. Current best value is 0.21722007003087074 with parameters: {'mx_depth': 80.0}.


In [7]:
best = {'_min': 0.0010076069969777687, '_max': 0.893393602554536, 'mx_depth': 80.0, 'mn_sp': 0.1550354937299351, 'mx_feat': 0.8211110248572838}

In [8]:
save_pkl('tuned_parameters/decison_tree_augmented.pkl', best)